# BERT Model

#### Goal of This File:

##### 1. Import Libraries, Helper Functions, and Constants ==> Data Sourcing and Munging ==> Utility Functions ==> Loading the Data ==> Merging all Individual Files

##### 2. Imbalanced Data Solution ==> Hyperparameters

##### 3. BERT

##### 4. Conclusion ==> Results ==> Tensorboard

## 1. Import Libraries, Helper Functions and Load Constants

In [ ]:
GDRIVE = True

In [2]:
if GDRIVE:
    !pip install pandas numpy nltk scikit-learn wordcloud\
    seaborn gensim tensorflow imblearn xgboost matplotlib unrar pyunpack more-itertools patool transformers > /dev/null

else:
    !pip install pandas numpy nltk scikit-learn wordcloud\
    seaborn gensim tensorflow imblearn xgboost matplotlib unrar pyunpack more-itertools patool transformers


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
!pip install pandas numpy nltk scikit-learn wordcloud\
seaborn gensim tensorflow imblearn xgboost matplotlib unrar pyunpack more-itertools patool transformers > /dev/null


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
VAST = True

if VAST:
    !sudo apt-get install unrar
    !sudo apt-get install rar
    

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.6.6-2build1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
rar is already the newest version (2:5.5.0-1build1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.


In [5]:
UTILS_URL = 'https://f005.backblazeb2.com/file/gp-support-files/nlp_ai_utils.py'
UPDATING_VALUES_URL = 'https://f005.backblazeb2.com/file/gp-support-files/updating_values.py'
ALL_LIBS_URL = 'https://f005.backblazeb2.com/file/gp-support-files/all_libs_dl.py'
CHUNKS_URLS_FILE_URL = 'https://f005.backblazeb2.com/file/gp-support-files/chunks_urls.py'

UTILS_FILE_NAME = 'nlp_ai_utils'
UPDATING_VALUES_FILE_NAME = 'updating_values'
ALL_LIBS_FILE_NAME = 'all_libs_dl'
CHUNKS_URLS_FILE_NAME = 'chunks_urls'

DEP_FILE_EXT = '.py'

In [6]:
import requests
import os

def get_dependencies(url,file_name,file_extension):
    if os.path.exists(file_name + file_extension):
        return print(file_name + " already exists.")
    else:
        print(f"downloading {file_name}...")
        r = requests.get(url)
        with open(file_name + file_extension, 'wb') as fd:
            for chunk in r.iter_content():
                fd.write(chunk)

In [7]:
get_dependencies(UTILS_URL,UTILS_FILE_NAME,DEP_FILE_EXT)
get_dependencies(UPDATING_VALUES_URL,UPDATING_VALUES_FILE_NAME,DEP_FILE_EXT)
get_dependencies(ALL_LIBS_URL,ALL_LIBS_FILE_NAME,DEP_FILE_EXT)
get_dependencies(CHUNKS_URLS_FILE_URL,CHUNKS_URLS_FILE_NAME,DEP_FILE_EXT)

nlp_ai_utils already exists.
updating_values already exists.
all_libs_dl already exists.
chunks_urls already exists.


In [8]:
from nlp_ai_utils import *
from chunks_urls import CHUNKS_URLS
from updating_values import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2023-06-05 15:01:58.324020: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
TF_ENABLE_ONEDNN_OPTS = 0
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
USE_READY_DATA = False
LIMIT = 60
DATA_URLS = CHUNKS_URLS[:LIMIT]

if USE_READY_DATA:
    READY_DATA_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/ready_for_models.rar'
    CLASS_WEIGHTS_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/class_weights.pickle'
    UNIQUE_WORDS_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/unique_words.pickle'
else:
    READY_DATA_URL = False
    CLASS_WEIGHTS_URL = False
    UNIQUE_WORDS_URL = False

if GDRIVE:
    PICKLES_DIR = "pickle_files"
    READY_DATASET_PATH = "larger_dataset/ready_for_models/"
    LARGER_DATASET_PATH = "larger_dataset"
    #GLOVE_FILES_DIR = "glove_files"
    SAVED_MODELS_DIR = 'saved_models'
else:
    PICKLES_DIR = "../pickle_files"    
    LARGER_DATASET_PATH = "../larger_dataset"
    READY_DATASET_PATH = "../larger_dataset/ready_for_models/"
    #GLOVE_FILES_DIR = "../glove_files"
    SAVED_MODELS_DIR = '../saved_models'
    
PREPROCESSED_CHUNKS_PATH = LARGER_DATASET_PATH + "/preprocessed_data_chunks/"
BASE_FILE_NAME = "chunk_"
FILE_FORMAT = ".csv"
DATA_IN_CHUNK = 116505
if LIMIT == len(CHUNKS_URLS):
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT + 20
else:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT
    
RANDOM_STATE = CONST_RANDOM_STATE
DELETE_PICKLES_AFTER_TRAINING = False
BALANCE_DATA = True
USE_MODIFIED_LABELS = True

### 1.1 Data Sourcing and Munging

#### 1.1.1 Loading The Data

In [10]:
if not USE_READY_DATA:
    if not os.path.exists(LARGER_DATASET_PATH):
        os.mkdir(LARGER_DATASET_PATH)
    if not os.path.exists(PREPROCESSED_CHUNKS_PATH):
        os.mkdir(PREPROCESSED_CHUNKS_PATH)
    get_chunks(DATA_URLS,LIMIT,1,BASE_FILE_NAME,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

chunk_1 already exists.
chunk_2 already exists.
chunk_3 already exists.
chunk_4 already exists.
chunk_5 already exists.
chunk_6 already exists.
chunk_7 already exists.
chunk_8 already exists.
chunk_9 already exists.
chunk_10 already exists.
chunk_11 already exists.
chunk_12 already exists.
chunk_13 already exists.
chunk_14 already exists.
chunk_15 already exists.
chunk_16 already exists.
chunk_17 already exists.
chunk_18 already exists.
chunk_19 already exists.
chunk_20 already exists.
chunk_21 already exists.
chunk_22 already exists.
chunk_23 already exists.
chunk_24 already exists.
chunk_25 already exists.
chunk_26 already exists.
chunk_27 already exists.
chunk_28 already exists.
chunk_29 already exists.
chunk_30 already exists.
chunk_31 already exists.
chunk_32 already exists.
chunk_33 already exists.
chunk_34 already exists.
chunk_35 already exists.
chunk_36 already exists.
chunk_37 already exists.
chunk_38 already exists.
chunk_39 already exists.
chunk_40 already exists.
chunk_41 

#### 1.1.2 Merging all Individual Files

In [11]:
#get all names of downloaded files
if not USE_READY_DATA:
    all_file_names = get_all_file_names(BASE_FILE_NAME,LIMIT)

In [12]:
#read all chunks into a list
if not USE_READY_DATA:
    list_dfs = read_chunks(all_file_names,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

In [13]:
#concatenate all chunks into a singular df
if not USE_READY_DATA:
    df = group_up_chunks(list_dfs)

In [14]:
#check how much of the data was actually downloaded
if not USE_READY_DATA:
    percent_loaded = check_no_missing_data(df.shape[0],ACTUAL_DATA_SHAPE)
    percent_loaded

In [15]:
if not USE_READY_DATA:
    review_data = df[['text', 'stars']].copy()

In [16]:
if not USE_READY_DATA:
    review_data.reset_index(inplace = True)
    review_data.drop(['index'],axis = 1,inplace = True)

In [17]:
if not USE_READY_DATA:
    review_data.rename(columns = {'text':'full_review_text','stars':'star_rating'}, inplace = True)

In [18]:
if not USE_READY_DATA:
    review_data.isnull().sum()

In [19]:
if not USE_READY_DATA:
    review_data.dropna(inplace = True)

In [20]:
if not USE_READY_DATA:
    review_data['full_review_text'].replace('', np.nan, inplace=True)
    review_data.dropna(inplace = True)

## 2. Imbalanced Data Solution

In [21]:
if BALANCE_DATA:
    if not USE_MODIFIED_LABELS:
        label_1 = review_data[review_data['star_rating'] == 1]
        label_2 = review_data[review_data['star_rating'] == 2]
        label_3 = review_data[review_data['star_rating'] == 3]
        label_4 = review_data[review_data['star_rating'] == 4]
        label_5 = review_data[review_data['star_rating'] == 5]

        minority_class = min([label_1.shape[0],label_2.shape[0],label_3.shape[0],label_4.shape[0],label_5.shape[0]])
    else:
        label_1 = review_data[review_data['star_rating'] == 1]
        label_2 = review_data[review_data['star_rating'] == 2]
        label_3 = review_data[review_data['star_rating'] == 3]
        label_4 = review_data[review_data['star_rating'] == 4]
        label_5 = review_data[review_data['star_rating'] == 5]

        minority_class = label_3.shape[0] / 2

In [22]:
if BALANCE_DATA:
    if USE_MODIFIED_LABELS:
        review_data_label_1 = label_1[:int(minority_class)]
        review_data_label_2 = label_2[:int(minority_class)]
        review_data_label_3 = label_3
        review_data_label_4 = label_4[:int(minority_class)]
        review_data_label_5 = label_5[:int(minority_class)]

    else:
        review_data_label_1 = label_1[:int(minority_class)]
        review_data_label_2 = label_2[:int(minority_class)]
        review_data_label_3 = label_3[:int(minority_class)]
        review_data_label_4 = label_4[:int(minority_class)]
        review_data_label_5 = label_5[:int(minority_class)]

    review_data = pd.concat([review_data_label_1,review_data_label_2,review_data_label_3,review_data_label_4,review_data_label_5])

    print(review_data.shape)

    print(review_data['star_rating'].value_counts())

(2075683, 2)
star_rating
3.0    691895
1.0    345947
2.0    345947
4.0    345947
5.0    345947
Name: count, dtype: int64


In [23]:
if not USE_READY_DATA:
    X = review_data['full_review_text']
    y = review_data['star_rating']

In [24]:
if not USE_READY_DATA:
    X = pd.Series([str(text) for text in X])

In [25]:
if USE_MODIFIED_LABELS:
    translated_labels = translate_labels(y)
    y = pd.Series(encode_sent(translated_labels))

In [26]:
print(y.value_counts())

3    691895
2    691894
1    691894
Name: count, dtype: int64


In [27]:
if not USE_READY_DATA:
    y = y - 1

#### Classes' shapes before and after (if USE_MODIFIED_LABELS = True):

BEFORE: 1 ==> Very Bad ==> AFTER: 2 ==> Negative Sentiment

BEFORE: 2 ==> Bad ==> AFTER: 2  ==> Negative Sentiment

BEFORE: 3 ==> Ok ==> AFTER: 3  ==> Neutral Sentiment

BEFORE: 4 ==> Good ==> AFTER: 1  ==> Positive Sentiment

BEFORE: 5 ==> Very Good ==> AFTER: 1  ==> Positive Sentiment

THEN: 1 is subtracted from each label. So, the Labels go FROM: 2,3,1 TO: 1,2,0

#### Classes' shapes before and after (if USE_MODIFIED_LABELS = False):

1 is subtracted from each label. So, the Labels go FROM: 1,2,3,4,5 TO: 0,1,2,3,4

Why is 1 subracted from each label? To bring the data into the preferred shape of the class weights in both sklearn and keras (starting label is 0, not 1).

In [28]:
if not os.path.exists(PICKLES_DIR):
    os.mkdir(PICKLES_DIR)

In [29]:
if not USE_READY_DATA:
    if BALANCE_DATA:
        class_weights = False
    else:
        class_weights = compute_class_weight(class_weight = "balanced",classes = np.unique(y),y=y)
        class_weights = dict(zip(np.unique(y), class_weights))

### Hyperparameters

In [30]:
if USE_READY_DATA:
    get_chunks([UNIQUE_WORDS_URL],0,1,'unique_words',PICKLES_DIR + '/','.pickle',False)

In [31]:
if USE_READY_DATA:
    UNIQUE_WORDS = pickle.load(open(PICKLES_DIR + "/unique_words.pickle", "rb"))

### Hyperparameters

In [32]:
TRAIN_PERCENT = 0.8
ATTENTION_UNITS = 64
DENSE_UNITS = 1024
LABELS_COUNT = len(y.unique())
EMBEDDING_DIM = 100
MAX_TEXT_LEN = 250
CONV_FILTERS = 60
CONV_KERNEL_SIZE = 3
DROPOUT_VAL = 0.2
TRUNC_TYPE = 'post'
PADDING_TYPE = 'post'
OOV_TOKEN = "<OOV>"
BATCH_SIZE = 128
EPOCHS = 25

In [33]:
ACTUAL_DATA_SHAPE = len(X)

if BALANCE_DATA:
    temp = pd.DataFrame({'text':X,'labels':y})
    shuffled = temp.sample(frac=1,random_state=RANDOM_STATE)
    X = shuffled['text']
    y = shuffled['labels']

In [34]:
X.reset_index(drop=True,inplace=True)
y.reset_index(drop=True,inplace=True)

In [35]:
if not os.path.exists(READY_DATASET_PATH):
    os.mkdir(READY_DATASET_PATH)

In [36]:
if USE_READY_DATA:
    get_chunks([READY_DATA_URL],0,1,'ready_for_models',READY_DATASET_PATH,'.rar',False)
    Archive(os.path.join(READY_DATASET_PATH,"ready_for_models.rar")).extractall(READY_DATASET_PATH)

In [37]:
if USE_READY_DATA:
    train_set_padded = pickle.load(open(READY_DATASET_PATH + "train_set_padded.pickle", "rb"))
    train_labels = pickle.load(open(READY_DATASET_PATH + "train_labels.pickle", "rb"))
    valid_set_padded = pickle.load(open(READY_DATASET_PATH + "valid_set_padded.pickle", "rb"))
    validation_labels = pickle.load(open(READY_DATASET_PATH + "validation_labels.pickle", "rb"))
    test_set_padded = pickle.load(open(READY_DATASET_PATH + "test_set_padded.pickle", "rb"))
    test_labels = pickle.load(open(READY_DATASET_PATH + "test_labels.pickle", "rb"))
    EMBEDDING_LAYER = pickle.load(open(READY_DATASET_PATH + "EMBEDDING_LAYER.pickle", "rb"))

## 3. BERT

In [38]:
!pip install tensorflow_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 4.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [39]:
import tensorflow_hub as hub

In [40]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

2023-06-05 15:02:46.590088: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-05 15:02:46.601863: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-05 15:02:46.602223: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [41]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(DENSE_UNITS, activation='relu'))
model.add(tf.keras.layers.Dense(LABELS_COUNT, activation='softmax'))

2023-06-05 15:02:47.969721: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'sentences' with dtype string and shape [?]
	 [[{{node sentences}}]]
2023-06-05 15:02:47.974170: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]


In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 1024)              52224     
                                                                 
 dense_1 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 48,245,899
Trainable params: 48,245,899
Non-trainable params: 0
_________________________________________________________________


##### ==> here is a quick explaination of how the dataset will be split using a smaller sample example.
##### ==> dataset => 100
##### ==> train_set => tr_s (example: 80)
##### ==> valid_set => vs (example: 10)
##### ==> test_set => te_s (example: 10)
##### ==> t = tr_s (80) + vs (10)
##### ==> train_set = x[:80]
##### ==> valid_set = x[80:t]
##### ==> test_set = x[t:] why t? because => vs = ts

##### use this guideline if you are confused about how the train-validation-test split was done. Also, this is a future guide for me as well in case I forget.

* train_set_size = 6,990,280 * 0.8 = 5,592,224
* valid_set_size = 6,990,280 * 0.1 = 699,028
* train_plus_valid = 5,592,224 + 699,028 = 6,291,252

==> To Confirm: test_size = 6,990,280 - 6,291,252 = 699,028

* train_set = [0:5,592,224]
* train_labels = [0:5,592,224]
* validation_set = [5,592,224:6,291,252] ==> 699,028
* validation_labels = [5,592,224:6,291,252] ==> 699,028
* test_set = [6,291,252,6,990,280] ==> 699,028
* test_labels = [6,291,252,6,990,280] ==> 699,028

In [43]:
VALID_TEST_PERCENT = round(1 - TRAIN_PERCENT,1) / 2

In [44]:
TRAIN_SIZE = int(ACTUAL_DATA_SHAPE * TRAIN_PERCENT)
VALID_TEST_SIZE = int(ACTUAL_DATA_SHAPE * VALID_TEST_PERCENT)
TOTAL_TEST_SIZE = TRAIN_SIZE + VALID_TEST_SIZE

In [45]:
train_set = X[:TRAIN_SIZE]
train_labels = y[:TRAIN_SIZE]

valid_set= X[TRAIN_SIZE:TOTAL_TEST_SIZE]
valid_labels = y[TRAIN_SIZE:TOTAL_TEST_SIZE]

test_set = X[TOTAL_TEST_SIZE:]
test_labels = y[TOTAL_TEST_SIZE:]

In [46]:
train_set.reset_index(drop=True,inplace=True)
train_labels.reset_index(drop=True,inplace=True)

valid_set.reset_index(drop=True,inplace=True)
valid_labels.reset_index(drop=True,inplace=True)

test_set.reset_index(drop=True,inplace=True)
test_labels.reset_index(drop=True,inplace=True)

In [47]:
train_set = np.array(train_set)
train_labels = np.array(train_labels)

valid_set = np.array(valid_set)
valid_labels = np.array(valid_labels)

test_set = np.array(test_set)
test_labels = np.array(test_labels)

In [48]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs",histogram_freq=1)

In [50]:
metrics_callback = MetricsCallback(test_data = valid_set, y_true = valid_labels)

In [52]:
# Train the model
model.compile(optimizer=Adam(learning_rate=1e-5),loss=SparseCategoricalCrossentropy(),metrics=['accuracy'])
model.fit(train_set, train_labels,validation_data = (valid_set,valid_labels),\
                    batch_size=BATCH_SIZE, epochs=EPOCHS,\
                             callbacks=[tensorboard_callback,metrics_callback,\
                                        EarlyStopping(patience=8,min_delta=0.005,mode='min',verbose = 1,restore_best_weights=True,monitor='val_loss'),\
                                        ReduceLROnPlateau(monitor="val_loss",mode = 'min',factor=0.1,verbose = 1,min_delta=0.01,patience=4)])

Epoch 1/25


2023-06-05 15:04:10.346256: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall' with dtype float and shape [?,50]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall}}]]
2023-06-05 15:04:10.346342: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_3' with dtype int64 and shape [?]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_3}}]]
2023-06-05 15:04:11.494952: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used 

12974/12974 [==============================] - ETA: 0s - loss: 0.7589 - accuracy: 0.6600

2023-06-05 15:07:50.850455: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 11565744000 exceeds 10% of free system memory.


6487/6487 [==============================] - 12s 2ms/step
              precision    recall  f1-score   support

           0       0.76      0.78      0.77     68923
           1       0.74      0.76      0.75     69155
           2       0.61      0.57      0.59     69490

    accuracy                           0.71    207568
   macro avg       0.70      0.71      0.70    207568
weighted avg       0.70      0.71      0.70    207568

Macro Weighted F1-Score: 0.704422489049619
12974/12974 [==============================] - 239s 18ms/step - loss: 0.7589 - accuracy: 0.6600 - val_loss: 0.6613 - val_accuracy: 0.7059 - lr: 1.0000e-05
Epoch 2/25
12974/12974 [==============================] - ETA: 0s - loss: 0.6358 - accuracy: 0.7178

...

6487/6487 [==============================] - 13s 2ms/step
              precision    recall  f1-score   support

           0       0.80      0.80      0.80     68923
           1       0.77      0.79      0.78     69155
           2       0.63      0.62      0.63     69490

    accuracy                           0.73    207568
   macro avg       0.73      0.73      0.73    207568
weighted avg       0.73      0.73      0.73    207568

Macro Weighted F1-Score: 0.7327809161223784
12974/12974 [==============================] - 143s 11ms/step - loss: 0.6085 - accuracy: 0.7310 - val_loss: 0.6039 - val_accuracy: 0.7334 - lr: 1.0000e-05
Epoch 4/25
12974/12974 [==============================] - ETA: 0s - loss: 0.5946 - accuracy: 0.7379

2023-06-05 15:15:01.374358: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 11565744000 exceeds 10% of free system memory.


6487/6487 [==============================] - 13s 2ms/step
              precision    recall  f1-score   support

           0       0.80      0.80      0.80     68923
           1       0.78      0.78      0.78     69155
           2       0.64      0.63      0.63     69490

    accuracy                           0.74    207568
   macro avg       0.74      0.74      0.74    207568
weighted avg       0.74      0.74      0.74    207568

Macro Weighted F1-Score: 0.7380249339323697
12974/12974 [==============================] - 143s 11ms/step - loss: 0.5946 - accuracy: 0.7379 - val_loss: 0.5941 - val_accuracy: 0.7381 - lr: 1.0000e-05
Epoch 5/25
12974/12974 [==============================] - ETA: 0s - loss: 0.5857 - accuracy: 0.7422

2023-06-05 15:17:23.568400: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 11565744000 exceeds 10% of free system memory.


6487/6487 [==============================] - 13s 2ms/step
              precision    recall  f1-score   support

           0       0.81      0.80      0.80     68923
           1       0.78      0.78      0.78     69155
           2       0.64      0.64      0.64     69490

    accuracy                           0.74    207568
   macro avg       0.74      0.74      0.74    207568
weighted avg       0.74      0.74      0.74    207568

Macro Weighted F1-Score: 0.7409737794286436
12974/12974 [==============================] - 142s 11ms/step - loss: 0.5857 - accuracy: 0.7422 - val_loss: 0.5876 - val_accuracy: 0.7409 - lr: 1.0000e-05
Epoch 6/25
6487/6487 [==============================] - 13s 2ms/step
              precision    recall  f1-score   support

           0       0.81      0.80      0.80     68923
           1       0.78      0.79      0.78     69155
           2       0.64      0.64      0.64     69490

    accuracy                           0.74    207568
   macro avg       0.

In [53]:
if not os.path.exists(SAVED_MODELS_DIR):
    os.mkdir(SAVED_MODELS_DIR)

In [54]:
model.save_weights(SAVED_MODELS_DIR + "/bert_model" + ".h5")

In [60]:
pickle_out = open(SAVED_MODELS_DIR + "/bert_model_params" + ".pickle",'wb')
pickle.dump({'MAX_TEXT_LEN':MAX_TEXT_LEN,'BATCH_SIZE':BATCH_SIZE,'EPOCHS':EPOCHS,\
             'train_set_size':len(train_set),'optimizer':str(model.optimizer),\
             'learning_rate':str(model.optimizer.learning_rate),\
             'chunks_used':(len(train_set) + len(valid_set) + len(test_set)) // DATA_IN_CHUNK,\
             'LABELS_COUNT':LABELS_COUNT},pickle_out)
pickle_out.close()

## 4. Conclusion

In [61]:
if VAST:
    !tar -czf bert_model_logs.tar.gz logs

In [62]:
if DELETE_PICKLES_AFTER_TRAINING:
    if GDRIVE:
        !rm -r larger_dataset/ready_for_models
    else:
        !rm -r ../larger_dataset/ready_for_models

### 4.1 Results

In [64]:
results = model.evaluate(test_set, test_labels, batch_size=BATCH_SIZE)
print("test loss, test acc:", results)

1622/1622 [==============================] - 5s 3ms/step - loss: 0.5718 - accuracy: 0.7484
test loss, test acc: [0.5717654228210449, 0.7484402656555176]


### 4.2 Tensorboard

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs/fit